# 1. Import module

In [1]:
%run "C:\Users\shiwei\Documents\ImageAnalysis3\required_files\Startup_py3.py"
sys.path.append(r"C:\Users\shiwei\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

import pandas as pd

37372


# 2. Load Data relevant information

In [2]:
analysis_save_folder=r'\\10.245.74.158\Chromatin_NAS_8\Exported_data\20220402-Export_new_v2'

## 2.1 Get loci info from codebook

In [3]:
# Load sorted codebook (allows direct matrix slicing) with cell type info
# Load codebook (then sort to allow direct matrix slicing) with cell type info

celltype_codebook_fname = os.path.join(analysis_save_folder,'merged_codebook.csv')
celltype_codebook_df = pd.read_csv (celltype_codebook_fname, index_col=None)

# sort df temporailiy so matrix can be sliced by df order directly
#celltype_codebook_df = celltype_codebook_df.sort_values(by = ['chr_as_num', 'chr_order'], ignore_index=False)
celltype_codebook_df.head()

,Unnamed: 0,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,...,NDB_479,NDB_562,NDB_608,NDB_460,NDB_563,NDB_592,NDB_368,NDB_436,NDB_629,NDB_604
0,0,1:3742742-3759944,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1:6245958-6258969,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1:8740008-8759916,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,1:11247744-11257616,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,1:13741888-13757922,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
celltype_codebook_df['library']

0       CTP11
1       CTP11
2       CTP11
3       CTP11
4       CTP11
        ...  
1977    CTP13
1978    CTP13
1979    CTP13
1980    CTP13
1981    CTP13
Name: library, Length: 1982, dtype: object

In [5]:
celltype_codebook_df.columns.tolist()

['Unnamed: 0',
 'name',
 'id',
 'NDB_784',
 'NDB_755',
 'NDB_826',
 'NDB_713',
 'NDB_865',
 'NDB_725',
 'NDB_817',
 'NDB_710',
 'NDB_652',
 'NDB_971',
 'NDB_718',
 'NDB_743',
 'NDB_847',
 'NDB_683',
 'NDB_643',
 'NDB_815',
 'NDB_760',
 'NDB_965',
 'NDB_790',
 'NDB_917',
 'NDB_778',
 'NDB_728',
 'NDB_844',
 'NDB_974',
 'NDB_961',
 'NDB_746',
 'NDB_661',
 'NDB_875',
 'NDB_901',
 'NDB_773',
 'NDB_868',
 'NDB_812',
 'NDB_1027',
 'NDB_716',
 'NDB_754',
 'NDB_860',
 'NDB_856',
 'NDB_680',
 'NDB_634',
 'NDB_737',
 'NDB_715',
 'NDB_698',
 'NDB_883',
 'NDB_863',
 'NDB_1033',
 'NDB_878',
 'NDB_631',
 'NDB_758',
 'NDB_1075',
 'NDB_653',
 'NDB_1060',
 'NDB_905',
 'NDB_805',
 'NDB_866',
 'NDB_730',
 'NDB_887',
 'NDB_880',
 'NDB_752',
 'NDB_832',
 'NDB_896',
 'NDB_835',
 'NDB_989',
 'NDB_946',
 'NDB_641',
 'NDB_721',
 'NDB_899',
 'NDB_853',
 'NDB_941',
 'NDB_838',
 'NDB_647',
 'NDB_994',
 'NDB_881',
 'NDB_1066',
 'NDB_665',
 'NDB_637',
 'NDB_734',
 'NDB_706',
 'NDB_935',
 'NDB_889',
 'NDB_788',
 'ND

## 2.2 Make a new dataframe to store the transcirption annotation

In [6]:
# Get path for the py containing functions
import os
import sys
import importlib
module_path =r'C:\Users\shiwei\Documents\AnalysisTool_Chromatin'
if module_path not in sys.path:
    sys.path.append(module_path)
    
    # import gene selection modules
import gene_selection
importlib.reload(gene_selection)
import gene_to_loci
importlib.reload(gene_to_loci)

<module 'gene_to_loci' from 'C:\\Users\\shiwei\\Documents\\AnalysisTool_Chromatin\\gene_to_loci.py'>

In [7]:
#loci_df = pd.DataFrame(celltype_codebook_df['name'].tolist(), columns=['name'])
loci_df = celltype_codebook_df[['name','id','chr','chr_order','library','dtype']]
loci_df

,name,id,chr,chr_order,library,dtype
0,1:3742742-3759944,1,1,0.0,CTP11,combo
1,1:6245958-6258969,2,1,1.0,CTP11,combo
2,1:8740008-8759916,3,1,2.0,CTP11,combo
3,1:11247744-11257616,4,1,5.0,CTP11,combo
4,1:13741888-13757922,5,1,6.0,CTP11,combo
...,...,...,...,...,...,...
1977,X:101961397-101974103,987,X,36.0,CTP13,combo
1978,X:136808741-136818926,988,X,48.0,CTP13,combo
1979,X:140517829-140527051,989,X,50.0,CTP13,combo
1980,X:167157164-167167452,990,X,61.0,CTP13,combo


In [8]:
# Format the chr loci name and use as index
from gene_to_loci import loci_pos_format
loci_name_list = list(map(loci_pos_format, celltype_codebook_df['name'].tolist()))
loci_name_arr = np.array(loci_name_list)
loci_name_arr 

array([['chr1_3742742_3759944', '1', '3742742', '3759944'],
       ['chr1_6245958_6258969', '1', '6245958', '6258969'],
       ['chr1_8740008_8759916', '1', '8740008', '8759916'],
       ...,
       ['chrX_140517829_140527051', 'X', '140517829', '140527051'],
       ['chrX_167157164_167167452', 'X', '167157164', '167167452'],
       ['chrX_169963295_170005197', 'X', '169963295', '170005197']],
      dtype='<U25')

In [9]:
loci_df['loci_name'] = list(loci_name_arr[:,0])

loci_df['chr_order'] = loci_df['chr_order'].map(lambda x: int(x))

<ipython-input-9-1084f06fc461>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loci_df['loci_name'] = list(loci_name_arr[:,0])
<ipython-input-9-1084f06fc461>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loci_df['chr_order'] = loci_df['chr_order'].map(lambda x: int(x))


In [10]:
loci_df = loci_df.set_index ('loci_name')
loci_df

,name,id,chr,chr_order,library,dtype
loci_name,,,,,,
chr1_3742742_3759944,1:3742742-3759944,1,1,0,CTP11,combo
chr1_6245958_6258969,1:6245958-6258969,2,1,1,CTP11,combo
chr1_8740008_8759916,1:8740008-8759916,3,1,2,CTP11,combo
chr1_11247744_11257616,1:11247744-11257616,4,1,5,CTP11,combo
chr1_13741888_13757922,1:13741888-13757922,5,1,6,CTP11,combo
...,...,...,...,...,...,...
chrX_101961397_101974103,X:101961397-101974103,987,X,36,CTP13,combo
chrX_136808741_136818926,X:136808741-136818926,988,X,48,CTP13,combo
chrX_140517829_140527051,X:140517829-140527051,989,X,50,CTP13,combo


## 2.3 Get all gene annotation from the prepared Ensembl df

In [11]:
# load gene annotation (covering all genes from the SMART-seq) for chr locus
# this should be enough since we can only analyze the genes with info from SMARTseq
# also we used Ensembl for both gene2loci and loci2gene to be consistent 
pool_folder = r'L:\Shiwei\DNA_MERFISH_analysis\SMARTer_nuclei_MOp'
gene_annotation_df = pd.read_csv(os.path.join(pool_folder, "MOp_smart_sn_gene_chr_info_NEW_from_transcriptome_FORMAT.csv"),index_col=0)

gene_annotation_df.head()

,chr,start,end,gene_biotype,coding_strand,length,genomic_position
gene,,,,,,,
0610005C13Rik,7,45567794,45575327,antisense,-1,7533,chr7_45567794_45575327
0610009B22Rik,11,51685386,51688874,protein_coding,-1,3488,chr11_51685386_51688874
0610009E02Rik,2,26445696,26459390,processed_transcript,1,13694,chr2_26445696_26459390
0610009L18Rik,11,120348678,120351190,bidirectional_promoter_lncRNA,1,2512,chr11_120348678_120351190
0610010F05Rik,11,23564961,23633639,protein_coding,-1,68678,chr11_23564961_23633639


In [12]:
from gene_to_loci import find_genes_near_loci

loci_name = loci_df.index[100]
loci_name

sel_genes = find_genes_near_loci (loci_name, 
                          gene_annotation_df,
                          extend_dist = 500*1000, gene_coverage_type = 'tss')

sel_genes

'Gm13582; Psmd14; Rbms1; Tank'

In [13]:
# Function to find genes near a list of loci provided in DataFrame 
from gene_to_loci import find_genes_near_loci

def find_genes_near_loci_for_loci_dataframe (loci_df:pd.core.frame.DataFrame, 
                                             gene_annotation_df:pd.core.frame.DataFrame, 
                                             extend_dist = 50*1000,
                                             gene_coverage_type = 'tss',
                                             key_added = None):


    """Find genes near loci for a list of loci provided in DataFrame"""
    sel_genes_list = []
    for loci_name in loci_df.index.tolist():
        sel_genes =  find_genes_near_loci (loci_name, 
                                           gene_annotation_df,
                                           extend_dist = extend_dist, 
                                           gene_coverage_type = gene_coverage_type
                                          )
        sel_genes_list.append(sel_genes)
    
    if isinstance (key_added, type(None)):
        extend_dist_name = str(int(extend_dist/1000)) + 'kb'
        key_added = f'adjacent_genes_{extend_dist_name}_{gene_coverage_type}'
    
    loci_df[key_added]=sel_genes_list
    
    return loci_df

In [14]:
loci_df_copy = loci_df.copy()

for _dist in [0*1000,10*1000,50*1000,100*1000,200*1000,500*1000,1000*1000,1500*1000,2000*1000,]:
    print(f'Add gene annotation for {_dist}bp.')
    loci_df_copy = find_genes_near_loci_for_loci_dataframe (loci_df_copy, 
                                            gene_annotation_df,
                                             extend_dist = _dist,
                                             gene_coverage_type = 'tss',
                                             key_added = None)

loci_df_copy

Add gene annotation for 0bp.
Add gene annotation for 10000bp.
Add gene annotation for 50000bp.
Add gene annotation for 100000bp.
Add gene annotation for 200000bp.
Add gene annotation for 500000bp.
Add gene annotation for 1000000bp.
Add gene annotation for 1500000bp.
Add gene annotation for 2000000bp.


,name,id,chr,chr_order,library,dtype,adjacent_genes_0kb_tss,adjacent_genes_10kb_tss,adjacent_genes_50kb_tss,adjacent_genes_100kb_tss,adjacent_genes_200kb_tss,adjacent_genes_500kb_tss,adjacent_genes_1000kb_tss,adjacent_genes_1500kb_tss,adjacent_genes_2000kb_tss
loci_name,,,,,,,,,,,,,,,
chr1_3742742_3759944,1:3742742-3759944,1,1,0,CTP11,combo,intergenic,intergenic,intergenic,Xkr4,Xkr4,Gm18956; Gm7341; Xkr4,Gm18956; Gm2053; Gm6085; Gm6101; Gm6119; Gm734...,Atp6v1h; Gm16041; Gm17101; Gm18956; Gm2053; Gm...,Atp6v1h; Gm16041; Gm17101; Gm18956; Gm2053; Gm...
chr1_6245958_6258969,1:6245958-6258969,2,1,1,CTP11,combo,intergenic,intergenic,4732440D04Rik; Rb1cc1,4732440D04Rik; Rb1cc1,4732440D04Rik; Gm19026; Gm2147; Rb1cc1,4732440D04Rik; Gm19026; Gm2147; Npbwr1; Rb1cc1...,4732440D04Rik; Gm19026; Gm2147; Gm5694; Gm7182...,4732440D04Rik; Atp6v1h; Gm16041; Gm17101; Gm19...,4732440D04Rik; Atp6v1h; Gm16041; Gm17101; Gm19...
chr1_8740008_8759916,1:8740008-8759916,3,1,2,CTP11,combo,intergenic,intergenic,Gm15452,Gm15452,Gm15452,Gm15452,1700034P13Rik; 2610203C22Rik; Adhfe1; Gm15452;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...
chr1_11247744_11257616,1:11247744-11257616,4,1,5,CTP11,combo,intergenic,intergenic,intergenic,intergenic,A830018L16Rik,A830018L16Rik; Prex2,A830018L16Rik; Cpa6; Gm15603; Gm15604; Gm7560;...,A830018L16Rik; Arfgef1; Cops5; Cpa6; Cspp1; Gm...,1700034P13Rik; 2610203C22Rik; A830018L16Rik; A...
chr1_13741888_13757922,1:13741888-13757922,5,1,6,CTP11,combo,intergenic,intergenic,Gm5523,Gm5523; Lactb2,Gm5523; Lactb2; Tram1,Gm5523; Gm7593; Lactb2; Ncoa2; Tram1,Eya1; Gm5523; Gm7593; Lactb2; Msc; Ncoa2; Slco...,Eya1; Gm17644; Gm2383; Gm5523; Gm6216; Gm7593;...,Eya1; Gm10566; Gm17644; Gm2383; Gm5523; Gm6216...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrX_101961397_101974103,X:101961397-101974103,987,X,36,CTP13,combo,intergenic,intergenic,intergenic,intergenic,8030474K03Rik; Ercc6l; Gm9095; H3f3a-ps1; Nhsl...,8030474K03Rik; Cited1; Ercc6l; Gm14843; Gm1484...,1700018G05Rik; 8030474K03Rik; Cited1; Dmrtc1a;...,1700018G05Rik; 4930519F16Rik; 8030474K03Rik; A...,1700018G05Rik; 4930519F16Rik; 8030474K03Rik; A...
chrX_136808741_136818926,X:136808741-136818926,988,X,48,CTP13,combo,intergenic,Plp1,Glra4; Plp1; Rab9b,BC065397; Glra4; Gm15026; Morf4l2; Plp1; Rab9b,BC065397; Glra4; Gm15026; Gm6275; Morf4l2; Plp...,BC065397; Fam199x; Glra4; Gm14994; Gm15026; Gm...,5730412P04Rik; Arxes1; Arxes2; BC065397; Bex1;...,5730412P04Rik; Armcx5; Arxes1; Arxes2; BC06539...,3632454L22Rik; 5730412P04Rik; Armcx2; Armcx5; ...
chrX_140517829_140527051,X:140517829-140527051,989,X,50,CTP13,combo,intergenic,intergenic,intergenic,Prps1; Tsc22d3,Frmpd3; Mid2; Prps1; Tsc22d3,Atg4a; Frmpd3; Gm6088; Gm7091; Mid2; Mir3475; ...,Atg4a; Col4a5; Col4a6; Frmpd3; Gm15013; Gm1507...,Atg4a; Col4a5; Col4a6; Frmpd3; Gm15013; Gm1507...,Acsl4; Atg4a; Col4a5; Col4a6; Frmpd3; Gm15000;...


In [16]:
# save the file 

loci_df_copy.to_csv(os.path.join(analysis_save_folder,'MERFISH_loci_adjacent_genes_tss_more_res.csv'))